In [12]:
import os
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import math
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
from tqdm import tqdm_notebook as tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)

In [13]:
structures = pd.read_csv('structures.csv')
mulliken = pd.read_csv('mulliken_charges.csv')

In [24]:
structures.head()

,molecule_name,atom_index,atom,x,y,z,mulliken_charge
0,dsgdb9nsd_000001,0,C,-0.013,1.086,0.008,-0.536
1,dsgdb9nsd_000001,1,H,0.002,-0.006,0.002,0.134
2,dsgdb9nsd_000001,2,H,1.012,1.464,0.000,0.134
3,dsgdb9nsd_000001,3,H,-0.541,1.448,-0.877,0.134
4,dsgdb9nsd_000001,4,H,-0.524,1.438,0.906,0.134


In [15]:
structures = structures.merge(mulliken, how = 'left', on = ['molecule_name', 'atom_index'])

In [21]:
structures_train = structures[pd.notna(structures['mulliken_charge'])]

In [23]:
for f in ['atom']:
    lbl = LabelEncoder()
    lbl.fit(list(structures_train[f].values) + list(structures_train[f].values))
    structures_train[f] = lbl.transform(list(structures_train[f].values))
    #df_test[f] = lbl.transform(list(df_test[f].values))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [27]:
X = structures_train.drop(['molecule_name', 'mulliken_charge'], axis=1)
Y = structures_train['mulliken_charge']
#X_test = df_test.drop(['id', 'molecule_name'], axis=1)

In [31]:
Y.describe()

count   1533537.000
mean         -0.000
std           0.225
min          -0.733
25%          -0.192
50%           0.099
75%           0.127
max           0.729
Name: mulliken_charge, dtype: float64

In [28]:
# Set up folds
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)

In [33]:
### Train the Model
params = {'num_leaves': 128,
          'min_child_samples': 79,
          'objective': 'regression',
          'n_estimators': 1000,
          'max_depth': 9,
          'learning_rate': 0.1,
          "boosting_type": "gbdt",
          "subsample_freq": 1,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'reg_alpha': 0.1,
          'reg_lambda': 0.3,
          'colsample_bytree': 1.0
         }

In [34]:
scores = []
for i, (train_index, test_index) in enumerate(kf.split(structures_train)):

        # Create data for this fold
        Y_train, Y_valid = Y.iloc[train_index].copy(), Y.iloc[test_index].copy()
        X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
        
        print( f'\nFold: {i}')

        fit_model = lgb.LGBMRegressor(**params)
        fit_model.fit(X_train, Y_train)
        pred = fit_model.predict(X_valid)
        # Save validation predictions for this fold
        print( "MAE: ", mean_absolute_error(Y_valid, pred))
        scores.append(mean_absolute_error(Y_valid, pred))
        
print(f'The mean score of a model for is: {np.mean(scores)}')


Fold: 0
MAE:  0.07748635037277962

Fold: 1
MAE:  0.07762656494992087

Fold: 2
MAE:  0.07786392247946096

Fold: 3
MAE:  0.0777743182533553

Fold: 4
MAE:  0.07733465539276016
The mean score of a model for is: 0.07761716228965539
